In [ ]:
INPUT_DATA_FOLDER = "lib"
INPUT_DATA_FILES = [
    #"a_example",
    #"b_read_on",
    #"c_incunabula",
    "d_tough_choices",
    #"e_so_many_books",
    #"f_libraries_of_the_world"
]
INPUT_DATA_EXT = ".txt"
OUTPUT_DATA_DIR = "out"
OUTPUT_DATA_EXT = ".out"
sep = "/"

def saveOut(fileLocation, data):
    with open(fileLocation, "w+") as f:
        f.write(data)

def parseOutputAndWriteToFile(registeredLibs):
    output = ""
    output += str(len(registeredLibs)) + "\n"

    for lib in registeredLibs:
        output += str(lib['id']) + " " + str(len(lib['books_Scanned'])) + "\n"
        for book in lib['books_Scanned']:
            output += str(book['book_id']) + ' '
        output += '\n'

    return output        
        
        
def importPS():
    for fileBase in INPUT_DATA_FILES:
        with open(INPUT_DATA_FOLDER + sep + fileBase + INPUT_DATA_EXT, "r") as f:
            outputFile = OUTPUT_DATA_DIR + sep + fileBase + OUTPUT_DATA_EXT

            meta, libraries = parsePS(f)
            registeredLibs = signupLibsAndScanBooks(meta, libraries)
            output = parseOutputAndWriteToFile(registeredLibs)
            saveOut(outputFile, output)



def sortLibsByValue(libraries):
    valuedLibs = []

    for lib in libraries:
        lib['libScore'] = \
                      (int(lib['booksShippingPerDay']) \
                       * int(lib['books_Count'])) \
                    - (int(lib['booksShippingPerDay']) \
                       * int(lib['signup_Days']))

        valuedLibs += [lib]

    sorted(valuedLibs, key = lambda libz: int(libz['libScore']))
    return valuedLibs


def signupLibsAndScanBooks(meta, libraries):

    libs = sortLibsByValue(libraries)

    registeredLibs = []
    inLibraryRegisteringProcess = False
    already_ScannedBook_Ids = []

    for day in range(int(meta['days']) - 27000):
        print("Day {}/{}".format(day, meta['days']))

        if not (len(registeredLibs)) == 0:

            if isRegistered(registeredLibs[-1], day):
                inLibraryRegisteringProcess = False

        if not inLibraryRegisteringProcess and len(libs) > 0:  
            inLibraryRegisteringProcess = True
            chosenLib = libs.pop(0)
            registeredLibs += [chosenLib]  
            chosenLib['signupStarted'] = day

        for lib in allLibsDoneRegistering(registeredLibs, day):
            for capacity in range(int(lib['booksShippingPerDay'])):
                bookToScan = getHighestValueBookNotYetScanned(lib, already_ScannedBook_Ids)

                if bookToScan is not None:

                    lib['books_Scanned'] += [bookToScan]
                    already_ScannedBook_Ids += [bookToScan['book_id']]

    return registeredLibs 


def getHighestValueBookNotYetScanned(lib, already_ScannedBook_Ids):
    for book in lib['book_Ids']:
        if not int(book['book_id']) in already_ScannedBook_Ids:
            lib['book_Ids'] = lib['book_Ids'][1:]
            return book
    return None


def allLibsDoneRegistering(registeredLibs, currentDay):
    successfullyRegisteredLibs = []

    for lib in registeredLibs:
        if isRegistered(lib, currentDay):
            successfullyRegisteredLibs += [lib]

    return successfullyRegisteredLibs


def isRegistered(lib, day):
    return int(lib['signupStarted']) + int(day) >= int(lib['signup_Days'])


def orderBooksByAndEnrichWithScore(book_Ids, metaData):
    possibleBooks = []

    for book in book_Ids:
        possibleBooks += [{'book_id': int(book), 'bookScore': metaData['bookScores'][int(book)]}]

    possibleBooks.sort(key = lambda b: b['bookScore'], reverse=True)

    if len(possibleBooks) > 1:
        return possibleBooks
    else:
        return []


def parsePS(problemFile):
    line1 = problemFile.readline().replace('\n', '')
    numberOfBooks, numberOfLibs, days = line1.split(' ')

    line2 = problemFile.readline().replace('\n', '')
    bookScores = line2.split(' ')
    meta = {'days': days, 'bookScores': bookScores}
    libraries = []

    for lib in range(int(numberOfLibs)):
        library = {'id': lib}
        lib1 = problemFile.readline().replace('\n', '').split(' ')
        lib2 = problemFile.readline().replace('\n', '').split(' ')
        library['books_Count'] = lib1[0]
        library['signup_Days'] = lib1[1]
        library['booksShippingPerDay'] = lib1[2]
        library['book_Ids'] = orderBooksByAndEnrichWithScore(lib2, meta)
        library['books_Scanned'] = []
        libraries += [library]

    return meta, libraries


if __name__ == "__main__":
    importPS()